In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [2]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [3]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [4]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [5]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday']
# contin_vars = []

In [6]:
from lightgbm import LGBMRegressor

In [7]:
y_out_columns = ['Sales']

In [8]:
# split_type = 'random'
# split_type = 'no_split'
split_type = 'last_week'

In [9]:
# Esta es para entrenar con todo
if split_type == 'no_split':
    df_train = df
elif split_type == 'last_week':
    # Esto divide en train y val
    df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
    df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')
elif split_type == 'random':
    # Splitting aleatorio
    np.random.seed(42)
    indexes = np.arange(len(df))
    np.random.shuffle(indexes)
    N = len(df)//5
    df_train = df[N:]
    df_val = df[:N]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [10]:
X_train = df_train[cat_vars + contin_vars]
if split_type != 'no_split':
    X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [11]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    if split_type != 'no_split':
        y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    if split_type != 'no_split':
        y_val = (df_val[y_out_columns].values - y_mean)/y_std

In [12]:
from sklearn.model_selection import cross_val_score
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

In [13]:
def calculate_RMSE(X, y, log_output=True):
    y_preds = np.exp(model.predict(X, verbose=1)*max_log_y)
    return np.sqrt((((y - y_preds)/y)**2).sum()/len(y_preds))

In [14]:
X_train.shape

(814150, 38)

In [15]:
def objective(params):
    params = {
        # 'max_depth': int(params['max_depth']),
        # 'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'learning_rate': '{:.4f}'.format(params['learning_rate'])
    }
    
    clf = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5, max_depth= -1,
        **params
    )
    
    score = -cross_val_score(clf, X_train.values, y_train.reshape(-1)).mean()
    return score

In [16]:
space = {
    # 'max_depth': hp.quniform('max_depth', 1, 64, 4),
    # 'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'learning_rate': hp.loguniform('learning_rate', -4, -2)
}

best = fmin(fn=objective,
            space=space,
            verbose=2,
            algo=tpe.suggest,
            max_evals=10)

100%|██████████| 10/10 [2:14:51<00:00, 809.17s/trial, best loss: -0.9018376829188288] 


In [17]:
best

{'learning_rate': 0.1276589493997595}

In [ ]:
clf = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5, max_depth= -1,
        **best
)

In [ ]:
cross_val_score(clf, X_train.values, y_train.reshape(-1)).mean()

In [ ]:
best.

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
parameters = {'learning_rate':[0.001, 0.05, 0.1, 0.5], 'max_depth':[1, 7, 31, 63]}
grid_search_res_1 = {'learning_rate': 0.05, 'min_child_samples': 5, 'max_depth': -1}
grid_search_res_2 = {'learning_rate': 0.05, 'max_depth': 7, 'min_child_samples': 20}

In [20]:
min_child_samples = 5
n_estimators = 4000
learning_rate = 0.1
model = LGBMRegressor(n_estimators=n_estimators, **grid_search_res_2)

In [21]:
clf = GridSearchCV(model, parameters, n_jobs=-1, verbose=1, cv=3)

In [22]:
fit_params={ "eval_metric" : 'l2', 
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars}
if split_type != 'no_split':
    fit_params["eval_set"] = [(X_val, y_val.reshape(-1))]
    fit_params["early_stopping_rounds"] = 100

In [23]:
# clf.fit(X_train, y_train.reshape(-1), **fit_params)
model.fit(X_train, y_train.reshape(-1), **fit_params)

C:\Users\Patricio\anaconda3\envs\curso\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\Patricio\anaconda3\envs\curso\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Assortment', 'CompetitionMonthsOpen', 'CompetitionOpenSinceYear', 'Day', 'DayOfWeek', 'Events', 'Month', 'Promo2SinceYear', 'Promo2Weeks', 'PromoInterval', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'SchoolHoliday_fw', 'State', 'StateHoliday', 'StateHoliday_bool_bw', 'StateHoliday_bool_fw', 'Store', 'StoreType', 'Week', 'Year']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Patricio\anaconda3\envs\curso\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Patricio\anaconda3\envs\cu

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 0.000282225
[200]	valid_0's l2: 0.000213215
[300]	valid_0's l2: 0.000173883
[400]	valid_0's l2: 0.000154846
[500]	valid_0's l2: 0.000143984
[600]	valid_0's l2: 0.000138124
[700]	valid_0's l2: 0.000133407
[800]	valid_0's l2: 0.000130175
[900]	valid_0's l2: 0.00012667
[1000]	valid_0's l2: 0.000124765
[1100]	valid_0's l2: 0.000123112
[1200]	valid_0's l2: 0.000121981
[1300]	valid_0's l2: 0.000121081
[1400]	valid_0's l2: 0.000120695
[1500]	valid_0's l2: 0.000120034
[1600]	valid_0's l2: 0.000119691
[1700]	valid_0's l2: 0.000119303
[1800]	valid_0's l2: 0.000118988
[1900]	valid_0's l2: 0.000118925
Early stopping, best iteration is:
[1814]	valid_0's l2: 0.000118869


LGBMRegressor(learning_rate=0.05, max_depth=7, n_estimators=4000)

In [24]:
clf

GridSearchCV(cv=3,
             estimator=LGBMRegressor(learning_rate=0.05, max_depth=7,
                                     n_estimators=4000),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.05, 0.1, 0.5],
                         'max_depth': [1, 7, 31, 63]},
             verbose=1)

In [25]:
# clf.best_params_

In [26]:
# clf.best_score_

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [27]:
model.score(X_val, y_val)

0.9167295341811057

In [28]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [29]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

0.1068625258147758

In [30]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

0.12346607713489312

In [31]:
calculate_RMSE(X_train, df_train['Sales'].values)

0.1068625258147758

In [32]:
calculate_RMSE(X_val, df_val['Sales'].values)

0.12346607713489312

# Sumbit a la competición

In [33]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_lightgbm_{split_type}-{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv', index=False)